In [1]:
import os
import glob

# Get the absolute path of the current project directory
project_dir = os.path.abspath('.')

# Get the parent of the parent directory
WORK_DIR = os.path.abspath(os.path.join(project_dir, '../../'))

# Change the working directory to the parent of the parent directory
os.chdir(WORK_DIR)

# Verify the change by printing the current working directory
print("Current Working Directory:", os.getcwd())

Current Working Directory: /Users/david.amat/Documents/david/pdf-search-llm-rag


# process_pdf_file

In [2]:
import PyPDF2
import pandas as pd
from typing import List

In [3]:
file_title = "attention_is_all_you_need"
pdf_path = f"data/{file_title}.pdf"
pdf_reader = PyPDF2.PdfReader(pdf_path)

In [4]:
pages_amount = len(pdf_reader.pages)
print(f'Amount of pages: {pages_amount}')

Amount of pages: 15


In [5]:
document_text = []

In [6]:
for i, page in enumerate(pdf_reader.pages):
    document_text.append([file_title, i+1, page.extract_text()])

In [7]:
# print(document_text[1][2])

In [8]:
columns = ["file_name", "page_number", "text"]
df_text = pd.DataFrame(document_text, columns=columns)

In [9]:
page_num = 1
start_text = document_text[page_num][2][:25]
end_text = document_text[page_num][2][-25:]

In [10]:
document_text[page_num][2]

'1 Introduction\nRecurrent neural networks, long short-term memory [ 13] and gated recurrent [ 7] neural networks\nin particular, have been firmly established as state of the art approaches in sequence modeling and\ntransduction problems such as language modeling and machine translation [ 35,2,5]. Numerous\nefforts have since continued to push the boundaries of recurrent language models and encoder-decoder\narchitectures [38, 24, 15].\nRecurrent models typically factor computation along the symbol positions of the input and output\nsequences. Aligning the positions to steps in computation time, they generate a sequence of hidden\nstates ht, as a function of the previous hidden state ht−1and the input for position t. This inherently\nsequential nature precludes parallelization within training examples, which becomes critical at longer\nsequence lengths, as memory constraints limit batching across examples. Recent work has achieved\nsignificant improvements in computational efficiency th

In [11]:
start_text

'1 Introduction\nRecurrent '

In [12]:
print(start_text)
print(end_text)

1 Introduction
Recurrent 
en generating the next.
2


# process_text_data

In [13]:
def split_page(document: str) -> List[str]:
    return document.split('.\n')

In [14]:
def get_paragraphs(data: pd.DataFrame) -> pd.DataFrame:
    # Explode the list to separate rows
    data_text_exploded = data.explode('text')

    # Add a 'paragraph' column indicating the index of the element in the list
    data_text_exploded['paragraph'] = data_text_exploded.groupby(
        ['file_name', 'page_number']
    ).cumcount() + 1

    return data_text_exploded

In [15]:
df = df_text.copy()

In [16]:
# Apply split_page function to split text into paragraphs
df['text'] = df['text'].apply(split_page)

In [17]:
for page_num in range(len(df['text'])):
    num_paragraphs = len(df['text'][page_num])
    print(f"Page {page_num}: {num_paragraphs} paragraphs")

Page 0: 6 paragraphs
Page 1: 12 paragraphs
Page 2: 5 paragraphs
Page 3: 7 paragraphs
Page 4: 12 paragraphs
Page 5: 8 paragraphs
Page 6: 8 paragraphs
Page 7: 9 paragraphs
Page 8: 7 paragraphs
Page 9: 13 paragraphs
Page 10: 22 paragraphs
Page 11: 18 paragraphs
Page 12: 4 paragraphs
Page 13: 6 paragraphs
Page 14: 6 paragraphs


In [18]:
data_text_exploded = df.explode('text')

In [19]:
data_text_exploded

,file_name,page_number,text
0,attention_is_all_you_need,1,"Provided proper attribution is provided, Googl..."
0,attention_is_all_you_need,1,Attention Is All You Need\nAshish Vaswani∗\nGo...
0,attention_is_all_you_need,1,∗Equal contribution. Listing order is random. ...
0,attention_is_all_you_need,1,†Work performed while at Google Brain
0,attention_is_all_you_need,1,‡Work performed while at Google Research
...,...,...,...
14,attention_is_all_you_need,15,<EOS>\n<pad>\nThe\nLaw\nwill\nnever\nbe\nperfe...
14,attention_is_all_you_need,15,<EOS>\n<pad>\nInput-Input Layer5\nThe\nLaw\nwi...
14,attention_is_all_you_need,15,<EOS>\n<pad>\nThe\nLaw\nwill\nnever\nbe\nperfe...
14,attention_is_all_you_need,15,<EOS>\n<pad>Figure 5: Many of the attention he...


In [20]:
data_text_exploded['paragraph'] = data_text_exploded.groupby(
    ['file_name', 'page_number']
).cumcount() + 1

In [21]:
data_text_exploded

,file_name,page_number,text,paragraph
0,attention_is_all_you_need,1,"Provided proper attribution is provided, Googl...",1
0,attention_is_all_you_need,1,Attention Is All You Need\nAshish Vaswani∗\nGo...,2
0,attention_is_all_you_need,1,∗Equal contribution. Listing order is random. ...,3
0,attention_is_all_you_need,1,†Work performed while at Google Brain,4
0,attention_is_all_you_need,1,‡Work performed while at Google Research,5
...,...,...,...,...
14,attention_is_all_you_need,15,<EOS>\n<pad>\nThe\nLaw\nwill\nnever\nbe\nperfe...,2
14,attention_is_all_you_need,15,<EOS>\n<pad>\nInput-Input Layer5\nThe\nLaw\nwi...,3
14,attention_is_all_you_need,15,<EOS>\n<pad>\nThe\nLaw\nwill\nnever\nbe\nperfe...,4
14,attention_is_all_you_need,15,<EOS>\n<pad>Figure 5: Many of the attention he...,5


In [22]:
# Apply get_paragraphs function to explode the list and add paragraph numbers
df = get_paragraphs(df)

In [23]:
df

,file_name,page_number,text,paragraph
0,attention_is_all_you_need,1,"Provided proper attribution is provided, Googl...",1
0,attention_is_all_you_need,1,Attention Is All You Need\nAshish Vaswani∗\nGo...,2
0,attention_is_all_you_need,1,∗Equal contribution. Listing order is random. ...,3
0,attention_is_all_you_need,1,†Work performed while at Google Brain,4
0,attention_is_all_you_need,1,‡Work performed while at Google Research,5
...,...,...,...,...
14,attention_is_all_you_need,15,<EOS>\n<pad>\nThe\nLaw\nwill\nnever\nbe\nperfe...,2
14,attention_is_all_you_need,15,<EOS>\n<pad>\nInput-Input Layer5\nThe\nLaw\nwi...,3
14,attention_is_all_you_need,15,<EOS>\n<pad>\nThe\nLaw\nwill\nnever\nbe\nperfe...,4
14,attention_is_all_you_need,15,<EOS>\n<pad>Figure 5: Many of the attention he...,5


In [24]:
# Apply strip to remove leading and trailing spaces
df['text'] = df['text'].str.strip()

In [25]:
# Reset index
df.reset_index(drop=True, inplace=True)

In [26]:
df = df[['file_name', 'page_number', 'paragraph', 'text']]

## Function

In [27]:
def process_text_data(df: pd.DataFrame) -> pd.DataFrame:
    # Apply split_page function to split text into paragraphs
    df['text'] = df['text'].apply(split_page)

    # Apply get_paragraphs function to explode the list and add paragraph numbers
    df = get_paragraphs(df)

    # Apply strip to remove leading and trailing spaces
    df['text'] = df['text'].str.strip()

    # Reset index
    df.reset_index(drop=True, inplace=True)

    # Reorder columns for better readability
    df = df[['file_name', 'page_number', 'paragraph', 'text']]

    return df

In [28]:
df = df_text.copy()
df_filtered = process_text_data(df)

In [29]:
df_filtered

,file_name,page_number,paragraph,text
0,attention_is_all_you_need,1,1,"Provided proper attribution is provided, Googl..."
1,attention_is_all_you_need,1,2,Attention Is All You Need\nAshish Vaswani∗\nGo...
2,attention_is_all_you_need,1,3,∗Equal contribution. Listing order is random. ...
3,attention_is_all_you_need,1,4,†Work performed while at Google Brain
4,attention_is_all_you_need,1,5,‡Work performed while at Google Research
...,...,...,...,...
138,attention_is_all_you_need,15,2,<EOS>\n<pad>\nThe\nLaw\nwill\nnever\nbe\nperfe...
139,attention_is_all_you_need,15,3,<EOS>\n<pad>\nInput-Input Layer5\nThe\nLaw\nwi...
140,attention_is_all_you_need,15,4,<EOS>\n<pad>\nThe\nLaw\nwill\nnever\nbe\nperfe...
141,attention_is_all_you_need,15,5,<EOS>\n<pad>Figure 5: Many of the attention he...


In [30]:
df_filtered.to_parquet("data/debug_read_pdf.parquet", engine="pyarrow")